In [1]:
# encoding:utf-8
import re
import pymysql
import requests
import MySQLdb
from bs4 import BeautifulSoup
from operator import itemgetter
from itertools import groupby
CONST_M = 20 # 定义常量m，在记录事件时使用
db = pymysql.connect("localhost","root","","DIPRE" ) # 连接数据库
cursor = db.cursor()
cursor.close()

In [2]:
def CrawlWeb(path):
    html = requests.get(path)
    soup = BeautifulSoup(html.text)
    body = soup.body
    return str(body) # 这里一定要注意转化成字符串，不然返回的是soup对象

def DelHrefInString(string):
    return re.sub('href="(.*?)"',"",string)

In [3]:
def GetData(): # 获取数据库中未处理的网页，以列表—字典的形式返回
    cursor = db.cursor()
    sql = "select * from webpage where mark = 0"
    data = []
    try:
        cursor.execute(sql)
        results = cursor.fetchall()
        for row in results:
            each = {}
            each['id'] = row[0]
            each['url'] = row[1]
            #each['text'] = DelHrefInString(CrawlWeb(row[1]))
            #each['url'] = row[1].lstrip('http://').lstrip('https://') # 截取掉http后获取URL
            data.append(each)
    except:
        db.rollback()
    cursor.close()
    return data

In [4]:
def GetSeeds(): # 获取数据库中的所有种子
    cursor = db.cursor()
    sql = "select * from seeds"
    seeds = []
    try:
        cursor.execute(sql)
        results = cursor.fetchall()
        for row in results:
            each = {}
            each['id'] = row[0]
            each['author'] = row[1]
            each['title'] = row[2]
            seeds.append(each)
    except:
        db.rollback()
    cursor.close()
    return seeds
seeds = GetSeeds()
seeds

[{'id': 20, 'author': 'Jonathan Swift', 'title': 'GULLIVERâ\x80\x99S TRAVELS'},
 {'id': 21, 'author': 'Charles Dickens', 'title': 'Pickwick Papers'},
 {'id': 22, 'author': 'Nicholas Sparks', 'title': 'The Last Song'},
 {'id': 23, 'author': 'Charles Dickens', 'title': 'Great Expectations'},
 {'id': 27, 'author': 'Adams, Richard', 'title': 'Plague Dogs'},
 {'id': 28,
  'author': 'Barry, John, Shepherd, Nigel',
  'title': 'Rock Climbing (Adventure Sports)'},
 {'id': 29,
  'author': 'David Hays, Daniel Hays',
  'title': 'My Old Man and the Sea: A Father and Son Sail Around Cape Horn'},
 {'id': 61, 'author': '冯唐', 'title': '无所畏'},
 {'id': 63, 'author': '李诞', 'title': '笑场'}]

In [5]:
def GetPatterns(): # 获取数据库中的所有模式
    cursor = db.cursor()
    sql = "select * from patterns"
    patterns = []
    try:
        cursor.execute(sql)
        results = cursor.fetchall()
        for row in results:
            each = {}
            each['id'] = row[0]
            each['order'] = row[1]
            each['urlprefix'] = row[2]
            each['prefix'] = row[3]
            each['middle'] = row[4]
            each['suffix'] = row[5]
            patterns.append(each)
    except:
        db.rollback()
    cursor.close()
    return patterns

In [6]:
def SaveSeed(author,title): # 将查找到的书籍元组存储到数据库中
    cursor = db.cursor()
    sql = 'INSERT INTO seeds(author,title) VALUES ("%s","%s")' % (author , title)
    try:
        cursor.execute(sql)
        db.commit()
        print("元组添加成功")
    except:
        db.rollback()
        print("元组添加失败")
    cursor.close()
    
def FindSeed(author,title):
    cursor = db.cursor()
    sql = 'select * from seeds where author = "%s" and title = "%s"' % (author,title)
    try:
        cursor.execute(sql)
        res = cursor.fetchone()
        if res is not None:
            return 1
        else:
            return 0
    except:
        db.rollback()
    return 0

def SavePattern(order,urlprefix,prefix,middle,suffix): # 将查找到的模式存储到数据库中
    cursor = db.cursor() # MySQLdb.escape_string()函数返回的字符串是byte类型的，需要重新解码成Unicode，还要双引号在外单引号在内
    sql = "INSERT INTO patterns(order_p,urlprefix,prefix,middle,suffix) VALUES (%d,'%s','%s','%s','%s')" % (order,MySQLdb.escape_string(urlprefix).decode('utf-8'),MySQLdb.escape_string(prefix).decode('utf-8'),MySQLdb.escape_string(middle).decode('utf-8'),MySQLdb.escape_string(suffix).decode('utf-8'))
#     print(MySQLdb.escape_string(urlprefix).decode('utf-8')) 
    print(sql)
    try:
        cursor.execute(sql)
        db.commit()
        print("模式添加成功")
    except:
        db.rollback()
        print("模式添加失败")
    cursor.close()

def FindPattern(order,urlprefix,prefix,middle,suffix):
    cursor = db.cursor()
    sql = "select * from patterns where order_p = %d and urlprefix ='%s' and prefix = '%s' and middle = '%s' and suffix = '%s'" % (order,urlprefix,prefix,middle,suffix)
    print(sql)
    try:
        cursor.execute(sql)
        res = cursor.fetchone()
        if res is not None:
            return 1
        else:
            return 0
    except:
        db.rollback()
    return 0

def MarkPage(id): # 通过ID标记一个网页已经搜索完成
    cursor = db.cursor()
    sql = "update webpage set mark=1 where id = %d" % (id)
    try:
        cursor.execute(sql)
        db.commit()
        print("标记修改成功")
    except:
        db.rollback()
        print("标记修改失败")
    cursor.close()

In [7]:
def FindBooksByPattern(pattern,text): # 通过一个模式在网页数据中查找元组并返回存储
    NewPattern = re.compile(pattern['prefix']+'(.*?)'+pattern['middle']+'(.*?)'+pattern['suffix'],re.I) # 忽略大小写进行查询
    m = NewPattern.finditer(text)
    if m is not None:
        for item in m:
            author = item.groups()[0].strip()
            title = item.groups()[1].strip()
            print(author,title)
            if FindSeed(author,title) == 0:
                SaveSeed(author,title)
        return 1 # 查找到了则返回1
    return None # 没有找到返回空

In [8]:
def SearchNearest(author,posTitle,string): # 查询文本中，取离title最近的author
    pattern = re.compile(author,re.I) # 忽略大小写进行查询
    m = pattern.finditer(string)
    minDistance = len(string)
    posAuthor = string.index(author)
    for item in m:
        dis = item.span()[0] - posTitle
        if dis < 0:
            dis = -dis
        if dis < minDistance:
            minDistance = dis
            posAuthor = item.span()[0]
    return posAuthor
# pos = SearchNearest('余华',6,data[1]['text'])
# print(data[1]['text'][pos-10:pos+10])

In [9]:
def GetMatchPattern(patterns,url): #查询某个网页是否有匹配的模式，返回匹配的第一个模式
    if patterns is None: # None不能进行迭代
        return None
    for item in patterns:
        if item['urlprefix'] in url:
            return item
    return None

In [10]:
def SearchSeedInPage(author,title,url,line): # 在文章中搜索一个种子的元组对并记录事件
    occurrence = {}
    if author in line and title in line: # 作者和标题都在该行则记录事件,需要先判定是作者在前还是标题在前
        posT = line.index(title)
#         posA = line.index(author)
        posA = SearchNearest(author,posT,line) # 搜索离标题最近的作者位置
        if(posA<posT): # 如果作者在前则为1，否则为0
            order = 1
            if posA-CONST_M>=0:
                prefix = line[posA-CONST_M:posA] # 记录author前m个字符作为前缀
            else:
                prefix = line[:posA] # 如果前m个字符超出下界则m记为0
            middle = line[posA+len(author):posT]
            suffix = line[posT+len(title):posT+len(title)+CONST_M] # 记录title后m个字符作为后缀，后界不用if，因为默认超出len按len计算
        else:
            order = 0
            if posT-CONST_M>=0:
                prefix = line[posT-CONST_M:posT]
            else:
                prefix = line[:posT]
            middle = line[posT+len(title):posA]
            suffix = line[posA+len(author):posA+len(author)+CONST_M]
        occurrence['author'] = author # 记录事件并返回
        occurrence['title'] = title
        occurrence['order'] = order
        occurrence['url'] = url
        occurrence['prefix'] = prefix.strip()
        occurrence['middle'] = middle.strip()
        occurrence['suffix'] = suffix.strip()
        return occurrence
    return None

In [11]:
def SearchSeedsInText(seeds,page): # 在一个数据中搜索所有种子
    occurrences = [] # 事件列表
    for seed in seeds:
        occurrence = SearchSeedInPage(seed['author'],seed['title'],page['url'],page['text'])
        if occurrence is not None:
            occurrences.append(occurrence)
    return occurrences

In [12]:
def WhetherFormPattern(occurrences): # 判断同个页面的事件集能否形成事件，即middle和order是否相同
    order = occurrences[0]['order']
    middle = occurrences[0]['middle']
    for item in occurrences:
        if order != item['order'] or middle != item['middle']:
            return 0
    return 1

def FormPattern(occurrences): # 利用查找到的事件形成模式，此时的事件集>2，且在同个url下
    prefix = occurrences[0]['prefix'] # 匹配数据中公共的前缀，不保证不为空
    suffix = occurrences[0]['suffix']
    pattern = {}
    
    for index in range(len(occurrences)-1,-1,-1):  # 这里的操作是默认前缀为第一个数据，然后与后面的数据进行匹配，在第一个数据的基础上做缩减操作
        i = len(prefix)-1
        j = len(occurrences[index]['prefix'])-1
        while i>=0 and j>=0 and occurrences[index]['prefix'][j] == prefix[i]:
            i = i-1
            j = j-1
        prefix = prefix[i+1:]
        
        m = 0
        n = 0
        while m<len(suffix) and n<len(occurrences[index]['suffix']) and occurrences[index]['suffix'][n] == suffix[m]:
            m = m+1
            n = n+1
        suffix = suffix[:m]
        
    pattern['order'] = occurrences[0]['order']
    pattern['middle'] = occurrences[0]['middle'].strip()
    pattern['urlprefix'] = occurrences[0]['url'].lstrip('http://').lstrip('https://').split('/')[0] # 去除http头，并取主网址，保存url
    pattern['prefix'] = prefix.strip()
    pattern['suffix'] = suffix.strip()
    if len(pattern['prefix']) and len(pattern['suffix']) and len(pattern['urlprefix']) and len(pattern['middle']): # 只返回符合特异性的模式
        return pattern
    return None

In [13]:
if __name__ == '__main__': 
    patterns = GetPatterns()
    data = GetData() # 获取所有没有被处理过的数据
    seeds = GetSeeds() # 获取所有种子
#     print(patterns,'\n')
#     print(data,'\n')
#     print(seeds,'\n')
    
    for index in range(len(data)-1,-1,-1): # 因为中途需要删除数据，所以从后往前遍历
        
        data[index]['text'] = DelHrefInString(CrawlWeb(data[index]['url']))
#         print(type(data[index]['text']))
        match_pattern = GetMatchPattern(patterns,data[index]['url']) # 对于每组数据，首先查找是否有现有匹配模式
        if match_pattern is not None: # 如果有匹配模式，则用匹配模式查询该数据（后续可以改为如果有多种匹配模式，则使用第一个能查找到结果的匹配模式）
            if FindBooksByPattern(match_pattern,data[index]['text']) is not None: # 如果利用匹配模式查询的结果不为空
                MarkPage(data[index]['id']) # 标记该网页已查询
                del data[index] # 在内存中删除处理过的数据
                continue # 处理下一个网页
                
#         # 如果没有匹配的模式，或者利用匹配模式查出来的结果为空，则进行以下步骤
        occurrences = SearchSeedsInText(seeds,data[index]) # 在该条数据中搜索所有种子对，并记录成事件集
        if occurrences is not None:
            if len(occurrences)<2: # 如果种子查询个数仅仅为1，则无法形成事件（后续改为存储到数据库中）
                continue
            if WhetherFormPattern(occurrences): # 如果事件集具有相同的middle和order则尝试形成模式
                pattern = FormPattern(occurrences)
                print(pattern)
                if pattern is not None: # 如果形成有效的模式，则利用模式在该页面下搜索
                    SavePattern(pattern['order'],pattern['urlprefix'],pattern['prefix'],pattern['middle'],pattern['suffix']) # 直接存储
                    FindBooksByPattern(pattern,data[index]['text']) # 通过形成的模式查找元组并存储
            MarkPage(data[index]['id']) # 处理完成后，删除该数据，并将网页标记为已搜索
            del data[index]

{'order': 1, 'middle': '"  target="_blank" title="', 'urlprefix': 'e.dangdang.com', 'prefix': 'v>\n</a>\n<a dd_name="', 'suffix': '">\n<span class="book'}
INSERT INTO patterns(order_p,urlprefix,prefix,middle,suffix) VALUES (1,'e.dangdang.com','v>\n</a>\n<a dd_name=\"','\"  target=\"_blank\" title=\"','\">\n<span class=\"book')
模式添加成功
兰陵笑笑生 金瓶梅(全两册)(崇祯版)(简体横排、无批评)
元组添加成功
蒙曼 蒙曼品最美唐诗：人生五味
元组添加成功
慈怀读书会 你最好的样子就是做自己
元组添加成功
蔡崇达 皮囊
元组添加成功
蒋方舟 东京一年
元组添加成功
慈怀读书会 把生活过成你想要的样子
元组添加成功
上彊村民 宋词三百首(作家榜经典文库，马未都推荐版！直抵生活美学的源头，遇见内心向往的生活！全新未删节插图珍藏版)大星文化出品
元组添加成功
鲁迅 鲁迅自编文集（套装共21册）
元组添加成功
林清玄 在这坚硬的世界里,修得一颗柔软心
元组添加成功
（美）德博拉·海登,（英）乔治·奥威尔,鲁思·本尼迪克特 等 私家珍藏人文书库(套装共16册)
元组添加成功
贾平凹 自在独行【百万册精装纪念版】
元组添加成功
（日）村上春树 假如真有时光机
元组添加成功
乔瑞玲 董卿：做一个有才情的女子
元组添加成功
余华 余华：我们生活在巨大的差距里
元组添加成功
于谦 玩儿
元组添加成功
海子 海子的诗
元组添加成功
冯唐 无所畏
[日]村上春树(著);施小炜(译) 我的职业是小说家
元组添加成功
李诞 笑场
[美]阿尔伯特·哈伯德,[奥]西格蒙德·佛洛伊德,[奥]阿尔弗雷德·阿德勒,[清]曾国藩,[清]李鸿章 等 慢读?传世经典（套装共18册）
元组添加成功
标记修改成功


In [16]:
test = [{'url':'1.html'},{'url':'2.html'},{'url':'3.html'},{'url':'4.html'}] # 删除一个列表中的元组，则后面的元组自动往前
del test[2]
print(test[2])

{'url': '4.html'}
